In [1]:
import pandas as pd
import pm4py


# Konfigurationen 
FILE_PATH = "/Users/titusthamm/Library/Mobile Documents/com~apple~CloudDocs/Masterstudium/3. Semester/Praktikum/2. Assignment/BPI_SIMULATION/BPI Challenge 2017.xes"

In [2]:
log_df = pm4py.read_xes(FILE_PATH)
print(log_df.shape)

/Users/titusthamm/Library/Mobile Documents/com~apple~CloudDocs/Masterstudium/3. Semester/Praktikum/1. Assignment/.venv/lib/python3.13/site-packages/pm4py/utils.py:991: UserWarning: Install the optional requirement `rustxes` to import/export files faster.
  warnings.warn("Install the optional requirement `rustxes` to import/export files faster.")
/Users/titusthamm/Library/Mobile Documents/com~apple~CloudDocs/Masterstudium/3. Semester/Praktikum/1. Assignment/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 31509/31509 [00:17<00:00, 1847.93it/s]


(1202267, 19)


In [4]:
print(log_df.columns)

Index(['Action', 'org:resource', 'concept:name', 'EventOrigin', 'EventID',
       'lifecycle:transition', 'time:timestamp', 'case:LoanGoal',
       'case:ApplicationType', 'case:concept:name', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'OfferID'],
      dtype='object')


In [5]:
df = pm4py.convert_to_dataframe(log_df)

In [7]:
df.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#get the start event A_Create Application
arrivals = df[df['concept:name'] == 'A_Create Application'].copy()

# sorting it chronologically as we need the times between two neighboured instances
arrivals = arrivals.sort_values(by='time:timestamp')

# time differences between the preceeding and current trace
arrivals['inter_arrival_time'] = arrivals['time:timestamp'].diff()

# convert to Seconds
arrivals['seconds_gap'] = arrivals['inter_arrival_time'].dt.total_seconds()

# Drop the first row as it has no previous row
arrivals = arrivals.dropna(subset=['seconds_gap'])

# calculate mean and lambda for exponential distribution
mean_gap = arrivals['seconds_gap'].mean()
lambda_rate = 1 / mean_gap

print(f"Simulation Parameter (Mean Gap): {mean_gap:.2f} seconds")
print(f"Simulation Parameter (Lambda):   {lambda_rate:.6f} arrivals per second")

Simulation Parameter (Mean Gap): 1002.23 seconds
Simulation Parameter (Lambda):   0.000998 arrivals per second


In [11]:
plt.hist(arrivals['seconds_gap'], bins=100, range=(0, 3600)) # Inspecting gaps up to 1 hour
plt.title("Distribution of Arrival Gaps (Inter-Arrival Times)")
plt.xlabel("Gap in Seconds")
plt.ylabel("Frequency")
plt.show()

NameError: name 'plt' is not defined

In [10]:
arrivals.tail()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,inter_arrival_time,seconds_gap
1202068,Created,User_1,A_Create Application,Application,Application_1107993134,complete,2016-12-31 15:38:12.302000+00:00,Car,New credit,Application_1107993134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:02:31.131000,151.131
1202113,Created,User_1,A_Create Application,Application,Application_2103458006,complete,2016-12-31 16:40:45.514000+00:00,Existing loan takeover,New credit,Application_2103458006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 01:02:33.212000,3753.212
1202136,Created,User_1,A_Create Application,Application,Application_59622033,complete,2016-12-31 16:43:28.509000+00:00,Home improvement,New credit,Application_59622033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:02:42.995000,162.995
1202193,Created,User_1,A_Create Application,Application,Application_303923658,complete,2016-12-31 16:59:16.240000+00:00,Car,New credit,Application_303923658,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:47.731000,947.731
1202246,Created,User_1,A_Create Application,Application,Application_1350494635,complete,2016-12-31 21:37:53.216000+00:00,Home improvement,New credit,Application_1350494635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 04:38:36.976000,16716.976
